# Session 4: Advanced Topics & Real-World Considerations

## Learning Objectives

In this session, we'll explore several important topics through hands-on exercises:

1. **Train-Test Split Methods** - Random vs chronological splits and their impact on overfitting
2. **Execution Prices** - Asymmetric bid-ask spreads and realistic order book dynamics
3. **Benchmark Strategies** - Compare ML strategies with naive baselines

## Overview

These exercises highlight critical real-world considerations that can make or break a trading strategy. Understanding these concepts is essential for building robust systematic trading systems.


## Setup


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Import our modules
sys.path.insert(0, str(Path("..").resolve()))

from eda.analysis import basic_summary
from features.engineering import prepare_features, prepare_target
from backtesting.engine import backtest_strategy, print_backtest_metrics


In [ ]:
# Load Session 2/3 data
data_path = Path("../data/saved/stock_session2.csv")
df = pd.read_csv(data_path, parse_dates=["timestamp"])

print(f"Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# Prepare features and target
feature_cols = ["X1", "X2", "X3", "X4"]
X = prepare_features(df, feature_cols=feature_cols)
y_returns = prepare_target(df, target_col="returns")

# Binary classification target (for simplicity in some exercises)
y_direction = (y_returns > 0).astype(int)

print(f"\nFeatures: {feature_cols}")
print(f"Target: Returns (for regression) and Direction (for classification)")


## Exercise 1: Random Split vs Chronological Split

**The Problem:** With time-varying components (like X4 in our dataset), random train-test splits can give misleading results.

**What we'll do:**
- Train a Random Forest with both split methods
- Compare train/test performance
- See how random split masks overfitting
- Introduce walk-forward cross-validation


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Model to use
model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)

# Method 1: Chronological split (correct for time series)
split_idx = int(len(df) * 0.8)
X_train_chron = ...
X_test_chron = ...
y_train_chron = ...
y_test_chron = ...

model_chron = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
model_chron.fit(X_train_chron, y_train_chron)

train_pred_chron = model_chron.predict(X_train_chron)
test_pred_chron = model_chron.predict(X_test_chron)

train_acc_chron = accuracy_score(y_train_chron, train_pred_chron)
test_acc_chron = accuracy_score(y_test_chron, test_pred_chron)

print("=" * 80)
print("CHRONOLOGICAL SPLIT (Correct for Time Series)")
print("=" * 80)
print(f"Train Accuracy: {train_acc_chron:.4f}")
print(f"Test Accuracy: {test_acc_chron:.4f}")
print(f"Overfitting Gap: {train_acc_chron - test_acc_chron:.4f}")
print(f"\nTrain period: {df['timestamp'].iloc[0]} to {df['timestamp'].iloc[split_idx-1]}")
print(f"Test period: {df['timestamp'].iloc[split_idx]} to {df['timestamp'].iloc[-1]}")


In [ ]:
# Method 2: Random split (WRONG for time series, but commonly done)
X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(...
)

model_rand = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
model_rand.fit(X_train_rand, y_train_rand)

train_pred_rand = model_rand.predict(X_train_rand)
test_pred_rand = model_rand.predict(X_test_rand)

train_acc_rand = accuracy_score(y_train_rand, train_pred_rand)
test_acc_rand = accuracy_score(y_test_rand, test_pred_rand)

print("=" * 80)
print("RANDOM SPLIT (Incorrect for Time Series)")
print("=" * 80)
print(f"Train Accuracy: {train_acc_rand:.4f}")
print(f"Test Accuracy: {test_acc_rand:.4f}")
print(f"Overfitting Gap: {train_acc_rand - test_acc_rand:.4f}")
print("\n⚠️  WARNING: Random split mixes past and future data!")


In [ ]:
# Compare results

# Visualize difference of results between random and chronological split


### Walk-Forward Cross-Validation

For time series, we should use **walk-forward validation** (also called time-series cross-validation):

1. Train on data up to time $t$
2. Test on data from $t+1$ to $t+k$
3. Move forward and repeat

This simulates real trading where we only use past data to predict the future.


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Walk-forward cross-validation
tscv = TimeSeriesSplit(n_splits=5)
scores = []

# TODO implement Walk-forward Cross-Validation and compute test accuracy on each folds


In [ ]:
# Create asymmetric bid-ask based on expected returns
# If return is positive: bid closer to price (easier to buy)
# If return is negative: ask closer to price (easier to sell)

def create_asymmetric_bid_ask(df, base_spread=0.001, skew_factor=0.5):
    """
    Create asymmetric bid-ask spreads.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with price and returns
    base_spread : float
        Base spread (e.g., 0.001 = 0.1%)
    skew_factor : float
        How much to skew (0 = symmetric, 1 = maximum skew)
        
    Returns:
    --------
    pd.DataFrame with bid and ask columns
    """
    df = df.copy()
    
    # Predict future returns (we'll use actual returns for demonstration)
    # In practice, this would be model predictions
    expected_returns = df['returns'].shift(-1).fillna(0)  # Forward-looking for demo
    
    # Normalize expected returns to [-1, 1] range
    max_abs_return = expected_returns.abs().max()
    if max_abs_return > 0:
        normalized_returns = expected_returns / max_abs_return
    else:
        normalized_returns = expected_returns
    
    # Asymmetric spread: if positive return expected, bid is closer (easier to buy)
    # bid = price * (1 - spread/2 - skew * normalized_return * spread/2)
    # ask = price * (1 + spread/2 + skew * normalized_return * spread/2)
    
    df['bid_asym'] = df['price'] * (1 - base_spread/2 + skew_factor * normalized_returns * base_spread/2)
    df['ask_asym'] = df['price'] * (1 + base_spread/2 - skew_factor * normalized_returns * base_spread/2)
    
    # Symmetric (original)
    df['bid_sym'] = df['price'] * (1 - base_spread/2)
    df['ask_sym'] = df['price'] * (1 + base_spread/2)
    
    return df

# Create asymmetric bid-ask
df_with_spreads = create_asymmetric_bid_ask(df, base_spread=0.0002, skew_factor=1000)

# Visualize difference
sample_idx = slice(0, 100)
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Symmetric
axes[0].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['price'].iloc[sample_idx], 'k-', label='Price', linewidth=2)
axes[0].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['bid_sym'].iloc[sample_idx], 'b--', label='Bid (sym)', alpha=0.7)
axes[0].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['ask_sym'].iloc[sample_idx], 'r--', label='Ask (sym)', alpha=0.7)
axes[0].set_title('Symmetric Bid-Ask Spread')
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Asymmetric
axes[1].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['price'].iloc[sample_idx], 'k-', label='Price', linewidth=2)
axes[1].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['bid_asym'].iloc[sample_idx], 'b--', label='Bid (asym)', alpha=0.7)
axes[1].plot(df_with_spreads['timestamp'].iloc[sample_idx], 
             df_with_spreads['ask_asym'].iloc[sample_idx], 'r--', label='Ask (asym)', alpha=0.7)
axes[1].set_title('Asymmetric Bid-Ask Spread (Skewed Before Price Moves)')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Note: In asymmetric case, bid is closer to price when return is positive (easier to buy),")
print("and ask is closer when return is negative (easier to sell).")


Now let's create a modified backtesting function that uses bid/ask prices for execution.


In [ ]:
def backtest_with_bid_ask(df, predictions, initial_capital=100000, 
                          use_asymmetric=True, transaction_cost=0.0):
    """
    Backtest using bid/ask prices for execution.
    - Buy at ASK price (you pay the ask)
    - Sell at BID price (you receive the bid)
    """
    df = df.copy()
    df["prediction"] = predictions
    df["signal"] = np.where(df["prediction"] > 0, 1, -1)
    df["position"] = df["signal"].shift(1).fillna(0).astype(int)
    
    # Get execution prices
    bid_col = "bid_asym" if use_asymmetric else "bid_sym"
    ask_col = "ask_asym" if use_asymmetric else "ask_sym"
    
    # Track entry prices
    entry_price = None
    entry_position = 0
    
    df["strategy_returns"] = 0.0
    
    for i in range(1, len(df)):
        prev_pos = df["position"].iloc[i-1]
        curr_pos = df["position"].iloc[i]
        
        # Position change
        if prev_pos != curr_pos:
            # Close previous position
            if prev_pos == 1 and entry_price is not None:  # Close long
                exit_price = df[bid_col].iloc[i]
                df.loc[df.index[i], "strategy_returns"] = (exit_price / entry_price - 1) - transaction_cost
            elif prev_pos == -1 and entry_price is not None:  # Close short
                exit_price = df[ask_col].iloc[i]
                df.loc[df.index[i], "strategy_returns"] = (entry_price / exit_price - 1) - transaction_cost
            
            # Open new position
            if curr_pos == 1:  # Open long
                entry_price = df[ask_col].iloc[i]
                entry_position = 1
            elif curr_pos == -1:  # Open short
                entry_price = df[bid_col].iloc[i]
                entry_position = -1
            else:
                entry_price = None
                entry_position = 0
        elif curr_pos == 0:
            entry_price = None
            entry_position = 0
        # If holding, mark-to-market (simplified - use price returns)
        elif curr_pos != 0:
            if curr_pos == 1:
                df.loc[df.index[i], "strategy_returns"] = df["returns"].iloc[i]
            else:
                df.loc[df.index[i], "strategy_returns"] = -df["returns"].iloc[i]
    
    # Calculate equity
    df["equity"] = initial_capital * (1 + df["strategy_returns"]).cumprod()
    strategy_returns = df["strategy_returns"].dropna()
    total_return = (df["equity"].iloc[-1] / initial_capital - 1) * 100
    
    from backtesting.engine import calculate_sharpe_ratio, calculate_max_drawdown
    return {
        "equity_curve": df["equity"],
        "total_return_pct": total_return,
        "sharpe_ratio": calculate_sharpe_ratio(strategy_returns),
        "max_drawdown_pct": calculate_max_drawdown(df["equity"]) * 100,
        "win_rate_pct": (strategy_returns > 0).sum() / len(strategy_returns) * 100,
        "df": df
    }


In [ ]:
# Train a simple model for predictions
from sklearn.linear_model import LogisticRegression

split_idx = int(len(df) * 0.8)
X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_train = y_direction.iloc[:split_idx]
y_test = y_direction.iloc[split_idx:]

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_test_proba = model.predict_proba(X_test)[:, 1]

# Convert probabilities to return predictions
mean_positive = y_returns.iloc[split_idx:][y_returns.iloc[split_idx:] > 0].mean()
mean_negative = y_returns.iloc[split_idx:][y_returns.iloc[split_idx:] <= 0].mean()
y_pred_returns = np.where(y_test_proba >= 0.5, mean_positive, mean_negative)

# Test on test set
df_test = df_with_spreads.iloc[split_idx:].copy()

# Backtest with symmetric bid-ask
results_sym = backtest_with_bid_ask(df_test, y_pred_returns, use_asymmetric=False)

# Backtest with asymmetric bid-ask
results_asym = backtest_with_bid_ask(df_test, y_pred_returns, use_asymmetric=True)

# Compare
comparison = pd.DataFrame({
    'Symmetric Bid-Ask': [
        results_sym['total_return_pct'],
        results_sym['sharpe_ratio'],
        results_sym['max_drawdown_pct'],
        results_sym['win_rate_pct']
    ],
    'Asymmetric Bid-Ask': [
        results_asym['total_return_pct'],
        results_asym['sharpe_ratio'],
        results_asym['max_drawdown_pct'],
        results_asym['win_rate_pct']
    ]
}, index=['Total Return (%)', 'Sharpe Ratio', 'Max Drawdown (%)', 'Win Rate (%)'])

print("=" * 80)
print("SYMMETRIC vs ASYMMETRIC BID-ASK COMPARISON")
print("=" * 80)
print(comparison.round(4))
print("=" * 80)

# Visualize equity curves
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df_test['timestamp'], results_sym['equity_curve'], label='Symmetric Bid-Ask', linewidth=2)
ax.plot(df_test['timestamp'], results_asym['equity_curve'], label='Asymmetric Bid-Ask', linewidth=2)
ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value')
ax.set_title('Equity Curves: Symmetric vs Asymmetric Bid-Ask')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## Exercise 3: Benchmark Strategies

**The Problem:** Your ML strategy might look good, but is it better than simple baselines?

**What we'll do:**
- Compare logistic regression strategy with:
  1. **Buy-and-Hold**: Simple buy and never sell
  2. **Moving Average Crossover**: Classic technical indicator strategy
- Show that ML doesn't always beat simple strategies


In [ ]:
# Strategy 1: Buy-and-Hold
def buy_and_hold_backtest(df, initial_capital=100000):
    """Simple buy-and-hold strategy."""
    returns = 
    equity = 
    
    from backtesting.engine import calculate_sharpe_ratio, calculate_max_drawdown
    total_return = 
    sharpe = calculate_sharpe_ratio(returns)
    max_dd = calculate_max_drawdown(equity)
    
    return {
        'equity_curve': equity,
        'total_return_pct': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown_pct': max_dd * 100,
        'win_rate_pct': (returns > 0).mean() * 100
    }

# Strategy 2: Moving Average Crossover
def ma_crossover_backtest(df, short_window=10, long_window=50, initial_capital=100000):
    """
    Moving average crossover strategy.
    Buy when short MA crosses above long MA, sell when it crosses below.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['ma_short'] = 
    df['ma_long'] = 
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['ma_short'] > df['ma_long'], 'signal'] = 
    df.loc[df['ma_short'] < df['ma_long'], 'signal'] = 
    
    # Position (enter on signal change)
    df['position'] = df['signal'].shift(1).fillna(0).astype(int)
    
    # Calculate returns
    df['strategy_returns'] = df['position'] * df['returns']
    
    # Apply transaction costs
    position_changes = 
    df.loc[position_changes, 'strategy_returns'] -= 0.01  # 0.1% transaction cost
    
    # Equity curve
    df['equity'] = 
    
    # Metrics
    strategy_returns = 
    total_return = 
    
    from backtesting.engine import calculate_sharpe_ratio, calculate_max_drawdown
    sharpe = calculate_sharpe_ratio(strategy_returns)
    max_dd = calculate_max_drawdown(df['equity'])
    win_rate = (strategy_returns > 0).sum() / len(strategy_returns) * 100
    
    return {
        'equity_curve': df['equity'],
        'total_return_pct': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown_pct': max_dd * 100,
        'win_rate_pct': win_rate,
        'df': df
    }


In [ ]:
# Run all strategies on test set
df_test = df.iloc[split_idx:].copy()

# Buy-and-Hold
results_bh = buy_and_hold_backtest(df_test)

# Moving Average Crossover
results_ma = ma_crossover_backtest(df_test, short_window=10, long_window=50)

# Logistic Regression (from earlier)
results_ml = backtest_strategy(
    df_test,
    y_pred_returns,
    initial_capital=100000,
    transaction_cost=0.01,
    trade_at="close"
)

# Compare all strategies
comparison = pd.DataFrame({
    'Buy-and-Hold': [
        results_bh['total_return_pct'],
        results_bh['sharpe_ratio'],
        results_bh['max_drawdown_pct'],
        results_bh['win_rate_pct']
    ],
    'MA Crossover (10/50)': [
        results_ma['total_return_pct'],
        results_ma['sharpe_ratio'],
        results_ma['max_drawdown_pct'],
        results_ma['win_rate_pct']
    ],
    'Logistic Regression': [
        results_ml['total_return_pct'],
        results_ml['sharpe_ratio'],
        results_ml['max_drawdown_pct'],
        results_ml['win_rate_pct']
    ]
}, index=['Total Return (%)', 'Sharpe Ratio', 'Max Drawdown (%)', 'Win Rate (%)'])

print("=" * 80)
print("STRATEGY COMPARISON")
print("=" * 80)
print(comparison.round(4))
print("=" * 80)

# Visualize equity curves
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df_test['timestamp'], results_bh['equity_curve'], 
        label='Buy-and-Hold', linewidth=2, alpha=0.8)
ax.plot(df_test['timestamp'], results_ma['equity_curve'], 
        label='MA Crossover (10/50)', linewidth=2, alpha=0.8)
ax.plot(df_test['timestamp'], results_ml['equity_curve'], 
        label='Logistic Regression', linewidth=2, alpha=0.8)
ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value')
ax.set_title('Equity Curves: Strategy Comparison')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### Try Different MA Parameters

Let's see how sensitive the MA strategy is to parameters:


In [ ]:
# Test different MA parameters
ma_params = [
    (5, 20),
    (10, 50),
    (20, 100),
    (50, 200)
]

ma_results = []

# TODO play around with MA parameters and analyze backtest results


## Summary

In this session, we explored three critical topics:

1. ✅ **Train-Test Split Methods**: Random splits give misleading results with time-varying data. Always use chronological splits or walk-forward validation.

2. ✅ **Execution Prices**: Asymmetric bid-ask spreads reflect real market dynamics and significantly impact backtest performance. Always use realistic execution prices.

3. ✅ **Benchmark Strategies**: ML strategies must outperform simple baselines (buy-and-hold, MA crossover) to be worthwhile.

**Key Takeaways:**
- **Time series require special care**: Random splits don't work for financial data
- **Execution matters**: Bid-ask spreads and order book dynamics can make or break a strategy
- **Simplicity first**: If simple strategies work, use them. ML should add clear value.
- **Realistic backtesting**: Include all real-world constraints (spreads, costs, latency)

**Next Steps:**
- Walk-forward validation for robust model evaluation
- More sophisticated execution models (slippage, market impact)
- Portfolio-level considerations (multiple assets, risk management)
